<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2020SightVisit_varaious_CNNs_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sys

from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F


In [ ]:
# import cv2
# In case of google colab, uncomment the below line,
#from google.colab.patches import cv2_imshow

import PIL.Image as PILImage

# matplotlib はグラフ描画などに頻用される
import matplotlib.pyplot as plt

In [ ]:
# matplotlib の日本語化ライブラリのインストール
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
# このセルを実行するか，または，すぐ下のセルを実行して画像ファイルを `test.jpg` としてください
#!wget -O test.jpg https://image.shutterstock.com/image-photo/summer-portrait-cute-dalmatian-dog-600w-1124451662.jpg
#img_file = 'test.jpg'

In [ ]:
#ファイルをアップロードします
from google.colab import files
#files?

In [ ]:
files.upload()  # ご自身の PC からファイルをアップロードして下さい


In [ ]:
!mv 19960222_004.jpg test.jpg

In [ ]:
img_file = 'test.jpg'
img_pil = PILImage.open(img_file)
plt.title('PIL による画像表示例')
plt.axis(False); plt.imshow(img_pil)


In [ ]:
# どれか一つだけ，事前訓練済のモデルを使用する
import torchvision.models
model = torchvision.models.vgg16(pretrained=True)
#model = torchvision.models.squeezenet1_0(pretrained=True, progress=True)
#model = torchvision.models.densenet161(pretrained=True, progress=True)
#model = torchvision.models.inception_v3(pretrained=True, progress=True)
#model = torchvision.models.googlenet(pretrained=True, progress=True)
#model = torchvision.models.shufflenet_v2_x1_0(pretrained=True, progress=True)
#model = torchvision.models.mobilenet_v2(pretrained=True, progress=True)
#model = torchvision.models.resnext50_32x4d(pretrained=True, progress=True)
#model = torchvision.models.wide_resnet50_2(pretrained=True, progress=True)
#model = torchvision.models.mnasnet1_0(pretrained=True, progress=True)

In [ ]:
# PyTorch による画像変換，画像サイズの変換
t1 = transforms.Resize((224,224)) 

_img = t1(img_pil)
print(type(_img), np.array(_img).shape)  # <class 'PIL.Image.Image'> (224, 224, 3)

t2 = transforms.ToTensor()
_img = t2(_img)
print(type(_img), np.array(_img).shape)  # <class 'torch.Tensor'> (3, 224, 224)

t3 = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

_img = t3(_img)
print(type(_img), np.array(_img).shape)  # <class 'torch.Tensor'> (3, 224, 224)

In [ ]:
logit = model(_img.unsqueeze(0))
h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()

In [ ]:
import requests

LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'

# download the imagenet category list
classes = {int(key):value for (key, value)
          in requests.get(LABELS_URL).json().items()}


In [ ]:
n_best = 5 
outnp = logit.detach().numpy()
ids = np.argsort(-outnp[0])
for i in range(0, n_best):
    print('{:.3f} -> {}'.format(probs[i], classes[ids[i]]))